In [3]:
import csv

input_data = []

with open('violations.csv', 'r') as f:
    reader = csv.reader(f)

    for line in reader:
        input_data.append(line)

In [6]:
input_data[:4]

[['Summons Number',
  'Plate ID',
  'Registration State',
  'Plate Type',
  'Issue Date',
  'Violation Code',
  'Vehicle Body Type',
  'Vehicle Make',
  'Issuing Agency',
  'Street Code1',
  'Street Code2',
  'Street Code3',
  'Vehicle Expiration Date',
  'Violation Location',
  'Violation Precinct',
  'Issuer Precinct',
  'Issuer Code',
  'Issuer Command',
  'Issuer Squad',
  'Violation Time',
  'Time First Observed',
  'Violation County',
  'Violation In Front Of Or Opposite',
  'House Number',
  'Street Name',
  'Intersecting Street',
  'Date First Observed',
  'Law Section',
  'Sub Division',
  'Violation Legal Code',
  'Days Parking In Effect    ',
  'From Hours In Effect',
  'To Hours In Effect',
  'Vehicle Color',
  'Unregistered Vehicle?',
  'Vehicle Year',
  'Meter Number',
  'Feet From Curb',
  'Violation Post Code',
  'Violation Description',
  'No Standing or Stopping Violation',
  'Hydrant Violation',
  'Double Parking Violation'],
 ['4714702166',
  'KGK6659',
  'NY',
  'P